# Pipeline di Machine Learning per Cybersecurity

Questo notebook orchestra un'intera pipeline di machine learning per un task di classificazione di traffico di rete. Le fasi principali sono:

1.  **Generazione Dati**: Creazione di un dataset sintetico.
2.  **Preprocessing**: Pulizia e trasformazione dei dati.
3.  **Training**: Addestramento di un modello con grid search.
4.  **Predizione**: Utilizzo del modello per classificare un nuovo campione.

Tutta la logica è contenuta in moduli Python separati (`.py`) per mantere il codice pulito e riutilizzabile.

## Setup Iniziale

Per prima cosa, dobbiamo installare le librerie necessarie. Se stai eseguendo questo notebook in Google Colab, esegui la cella seguente. Se hai già caricato l'intero progetto (con i file `.py`), non dovresti aver bisogno di clonare il repository.

In [ ]:
# Installa le dipendenze necessarie
!pip install pandas scikit-learn tensorflow faker

## Fase 1: Generazione del Dataset Sintetico

Importiamo ed eseguiamo lo script che crea il nostro dataset di cybersecurity fittizio. Il file verrà salvato in `data/cybersecurity_data.csv`.

In [ ]:
from create_synthetic_data import generate_synthetic_data

generate_synthetic_data()

## Fase 2: Preprocessing dei Dati

Ora eseguiamo il modulo di preprocessing. Questo caricherà i dati, anonimizzerà gli IP e il target, e applicherà il one-hot encoding. Visualizzeremo un'anteprima dei dati processati.

In [ ]:
from preprocessing.process import preprocess_data

X, y, ip_map, target_map = preprocess_data()

if X is not None:
    print("\n--- Anteprima Feature (X) ---")
    display(X.head())
    print("\n--- Anteprima Target (y) ---")
    display(y.head())

## Fase 3: Training del Modello

Eseguiamo il training. Lo script eseguirà una grid search per trovare i migliori iperparametri, addestrerà diversi modelli e salverà il migliore in `models/best_model.keras`.

In [ ]:
from training.train import train_and_evaluate

training_results, best_model_path = train_and_evaluate()

## Fase 4: Predizione su un Nuovo Campione

Infine, usiamo il modello addestrato per classificare un nuovo campione di dati. Definiamo un campione fittizio e passiamolo alla funzione di predizione.

In [ ]:
from prediction.predict import predict

# Creiamo un campione di dati fittizio
sample = {
    "ip_sorgente": "192.168.1.10",
    "ip_destinazione": "10.0.0.5",
    "porta_sorgente": 12345,
    "porta_destinazione": 443,
    "protocollo": "UDP",
    "byte_inviati": 250,
    "byte_ricevuti": 1800,
    "pacchetti_inviati": 8,
    "pacchetti_ricevuti": 12,
}

prediction_label = predict(sample)

if prediction_label:
    print(f"\nRISULTATO FINALE: Il campione è stato classificato come: '{prediction_label}'")